<a href="https://colab.research.google.com/github/eltonaguiar/LoanerLab/blob/main/stock_beat_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install polygon-api-client==1.14.4

In [ ]:


import polygon
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
from typing import Dict, List, Optional
from scipy import stats
import logging
import asyncio
import aiohttp
import requests

# Configuration
CONFIG = {
    'STOCK_API_KEY': "2xiTaFKgrJA8eGyxd_tF5GTu3OTXMUWC",  # Polygon.io key
    'ALPHA_VANTAGE_KEY': "6XN7LYXEYUOIAM7M",  # Alpha Vantage key
    'TWELVE_DATA_KEY': "43e686519f7b4155a4a90eaae82fb63a",  # Twelve Data key
    'FINNHUB_KEY': "cvrl241r01qnpem84d0gcvrl241r01qnpem84d10",  # Finnhub key
    'STOCK_API_PROVIDER': "polygon",  # Default to Polygon, options: "alpha_vantage", "twelve_data", "finnhub"
    'STOCK_SYMBOLS': [
        "AAPL", "MSFT", "GOOGL", "AMZN", "META", "NVDA", "JPM", "V", "WMT", "JNJ",
        "PG", "MA", "HD", "BAC", "KO", "PEP", "XOM", "T", "VZ", "CSCO",
        "INTC", "CMCSA", "PFE", "UNH", "MRK", "ABT", "CVX", "DIS", "WFC", "CRM",
        "TSLA", "NFLX", "ADBE", "PYPL", "SQ", "AMD", "ORCL", "IBM", "QCOM", "TXN",
        "BA", "CAT", "GE", "MMM", "HON", "UPS", "FDX", "LOW", "NKE", "SBUX",
        "TGT", "COST", "LMT", "RTX", "GD", "GS", "MS", "C", "BLK", "SCHW",
        "DE", "UNP", "CSX", "NSC", "DAL", "UAL", "LUV", "AAL", "JBLU", "ALK",
        "GILD", "AMGN", "REGN", "VRTX", "BIIB", "MDT", "SYK", "BSX", "EW", "ZBH",
        "TMO", "DHR", "A", "ILMN", "IQV", "CVS", "CI", "ANTM", "HUM", "MCK",
        "WBA", "KR", "DG", "DLTR", "GIS", "KHC", "MDLZ", "CL", "KMB", "MO",
        "PM", "SO", "DUK", "D", "AEP", "EXC", "NEE", "ED", "PEG", "XEL",
        "EOG", "COP", "OXY", "PXD", "APA", "SLB", "HAL", "BKR", "FTI", "NOV",
        "ETR", "ES", "FE", "PPL", "SRE", "WEC", "AWK", "CMS", "LNT", "NI",
        "TROW", "BEN", "IVZ", "STT", "NTRS", "KEY", "CFG", "RF", "HBAN", "FITB",
        "SYF", "DFS", "ALL", "PGR", "TRV", "AIG", "MET", "PRU", "AFL", "LNC",
        "CB", "CINF", "WRB", "GL", "HIG", "L", "MKC", "HRL", "SJM", "CPB",
        "TSN", "CAG", "HSY", "CHD", "CLX", "EL", "TAP", "MNST", "K", "BF.B",
        "STZ", "ADM", "BG", "INGR", "LW", "POST", "FLO", "DAR", "PPC", "SEB",
        "EIX", "ATO", "CNP", "OGE", "PNW", "IDA", "POR", "HE", "BKH", "SWX",
        "ISRG", "DXCM", "IDXX", "RMD", "HOLX", "ALGN", "COO", "WST", "BAX", "BDX",
        "ABBV", "LLY", "BMY", "GSK", "NVS", "AZN", "SNY", "TAK", "ZTS", "JAZZ",
        "ACN", "FIS", "FISV", "GPN", "JKHY", "PAYX", "BR", "CTAS", "EFX", "EXPD",
        "WM", "RSG", "SRCL", "ROL", "TTC", "GWW", "FAST", "ODFL", "JBHT", "CHRW",
        "HUBB", "AOS", "ALLE", "MAS", "LII", "FBHS", "WHR", "NWL", "LEG", "TPX",
        "DOV", "ITW", "PH", "ROP", "AME", "CMI", "GNRC", "IR", "XYL", "FLS",
        "APD", "LIN", "SHW", "PPG", "ECL", "IFF", "ALB", "EMN", "FMC", "MOS",
        "CF", "NUE", "STLD", "RS", "ATI", "CRS", "CLF", "CMC", "AA", "X", "FCX",
        "NEM", "GOLD", "AEM", "KGC", "AU", "HL", "CDE", "EGO", "IAG", "PAAS",
        "WPM", "AGI", "BTG", "OR", "SAND", "SSRM", "SBSW", "HMY", "GFI", "AUY",
        "PVG", "EQX", "NGD", "MUX", "EXK", "SVM", "MAG", "FSM", "DRD", "SA",
        "TECK", "BHP", "RIO", "VALE", "SCCO", "MT", "PKX", "TX", "SID", "GGB",
        "HPQ", "HPE", "XRX", "MCHP", "AVGO", "SWKS", "ON", "TER", "ADI", "LRCX",
        "AMAT", "KLAC", "ASML", "SNPS", "CDNS", "ANSS", "NOW", "WDAY", "ADSK", "INTU",
        "PANW", "FTNT", "CHKP", "CRWD", "OKTA", "ZS", "SPLK", "DDOG", "NET", "MDB",
        "VEEV", "DOCU", "TEAM", "HUBS", "RNG", "TWLO", "ZEN", "U", "SNOW", "PLTR",
        "PATH", "BILL", "ASAN", "SMAR", "DBX", "BOX", "PCTY", "PAYC", "QLYS", "TTD"
    ],
    'CRYPTO_PAIRS': [
        {"id": "bitcoin", "symbol": "BTC/USD"},
        {"id": "ethereum", "symbol": "ETH/USD"},
        {"id": "binancecoin", "symbol": "BNB/USD"},
        {"id": "cardano", "symbol": "ADA/USD"},
        {"id": "solana", "symbol": "SOL/USD"}
    ],
    'RISK_FREE_RATE': 0.05,
    'MAX_HISTORY_DAYS': 30,
    'STOCK_API_CALLS_PER_MINUTE': 5,
    'CRYPTO_DAYS': 30,
    'TARGET_STOCKS': 15,
    'CRYPTO_ON': False
}

# Initialize logging
logging.basicConfig(
    filename='stock_analysis.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Initialize Polygon client
stock_client = polygon.RESTClient(CONFIG['STOCK_API_KEY'])

# Track stock API calls
last_stock_api_call_time = 0
stock_api_call_count = 0
stock_api_call_window_start = time.time()

# Store recommendation history and cumulative high-quality stocks
stock_recommendation_history = {}
crypto_recommendation_history = {}
all_stock_strong_buys = []
analyzed_symbols = set()

async def get_crypto_data(pair: Dict, days: int = 30) -> Optional[pd.DataFrame]:
    if not CONFIG['CRYPTO_ON']:
        return None
    symbol = pair['symbol']
    coin_id = pair['id']
    max_attempts = 3
    for attempt in range(1, max_attempts + 1):
        try:
            async with aiohttp.ClientSession() as session:
                url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart"
                params = {
                    'vs_currency': 'usd',
                    'days': days,
                    'interval': 'daily'
                }
                msg = f"Attempt {attempt}: Fetching data for {symbol} from CoinGecko..."
                print(msg)
                logging.info(msg)

                async with session.get(url, params=params) as response:
                    if response.status != 200:
                        raise Exception(f"HTTP {response.status}")
                    data = await response.json()

                if not data.get('prices'):
                    msg = f"No price data available for {symbol}"
                    print(msg)
                    logging.warning(msg)
                    return None

                prices = data['prices']
                volumes = data['total_volumes']
                df = pd.DataFrame({
                    'date': [datetime.utcfromtimestamp(p[0] / 1000) for p in prices],
                    'close': [p[1] for p in prices],
                    'volume': [v[1] for v in volumes]
                })

                if df.empty:
                    msg = f"Empty data received for {symbol}"
                    print(msg)
                    logging.warning(msg)
                    return None

                df.set_index('date', inplace=True)
                msg = f"Successfully fetched {len(df)} days of data for {symbol}"
                print(msg)
                logging.info(msg)
                return df

        except Exception as e:
            msg = f"Attempt {attempt} failed for {symbol}: {str(e)}"
            print(msg)
            logging.error(msg)
            if attempt < max_attempts:
                wait_time = 4 * (2 ** (attempt - 1))
                msg = f"Retrying in {wait_time} seconds..."
                print(msg)
                logging.info(msg)
                await asyncio.sleep(wait_time)
            else:
                msg = f"Max retries reached for {symbol}"
                print(msg)
                logging.error(msg)
                return None

def get_stock_data(symbol: str, days: int = 30) -> Optional[pd.DataFrame]:
    """Fetch stock data from the selected API provider with a 1-minute break"""
    global last_stock_api_call_time
    max_attempts = 3
    provider = CONFIG['STOCK_API_PROVIDER'].lower()

    for attempt in range(1, max_attempts + 1):
        try:
            current_time = time.time()
            time_since_last_call = current_time - last_stock_api_call_time
            if time_since_last_call < 60:
                wait_time = 60 - time_since_last_call
                msg = f"\nWaiting {wait_time:.1f} seconds for {symbol} to enforce 1-minute break..."
                print(msg)
                logging.info(msg)
                time.sleep(max(0, wait_time - (time.time() - current_time)))

            end_date = datetime.now()
            start_date = end_date - timedelta(days=days)
            from_date = start_date.strftime("%Y-%m-%d")
            to_date = end_date.strftime("%Y-%m-%d")

            msg = f"Attempt {attempt}: Fetching stock data for {symbol} from {provider} ({from_date} to {to_date})..."
            print(msg)
            logging.info(msg)

            if provider == "polygon":
                bars = stock_client.get_aggs(
                    ticker=symbol,
                    multiplier=1,
                    timespan="day",
                    from_=from_date,
                    to=to_date,
                    limit=days
                )
                if not bars:
                    msg = f"No data available for {symbol} from Polygon"
                    print(msg)
                    logging.warning(msg)
                    return None
                data = pd.DataFrame([{
                    'date': datetime.utcfromtimestamp(bar.timestamp / 1000),
                    'open': bar.open,
                    'high': bar.high,
                    'low': bar.low,
                    'close': bar.close,
                    'volume': bar.volume
                } for bar in bars if hasattr(bar, 'open') and hasattr(bar, 'close') and hasattr(bar, 'volume')])

            elif provider == "alpha_vantage":
                url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={CONFIG['ALPHA_VANTAGE_KEY']}&outputsize=compact"
                response = requests.get(url)
                data_json = response.json()
                if "Time Series (Daily)" not in data_json:
                    msg = f"No data available for {symbol} from Alpha Vantage: {data_json.get('Error Message', 'Unknown error')}"
                    print(msg)
                    logging.warning(msg)
                    return None
                data = pd.DataFrame([
                    {
                        'date': datetime.strptime(date, "%Y-%m-%d"),
                        'open': float(values["1. open"]),
                        'high': float(values["2. high"]),
                        'low': float(values["3. low"]),
                        'close': float(values["4. close"]),
                        'volume': int(values["5. volume"])
                    } for date, values in data_json["Time Series (Daily)"].items()
                ])

            elif provider == "twelve_data":
                url = f"https://api.twelvedata.com/time_series?symbol={symbol}&interval=1day&start_date={from_date}&end_date={to_date}&apikey={CONFIG['TWELVE_DATA_KEY']}"
                response = requests.get(url)
                data_json = response.json()
                if "values" not in data_json or not data_json["values"]:
                    msg = f"No data available for {symbol} from Twelve Data: {data_json.get('message', 'Unknown error')}"
                    print(msg)
                    logging.warning(msg)
                    return None
                data = pd.DataFrame([{
                    'date': datetime.strptime(v["datetime"], "%Y-%m-%d"),
                    'open': float(v["open"]),
                    'high': float(v["high"]),
                    'low': float(v["low"]),
                    'close': float(v["close"]),
                    'volume': int(v["volume"])
                } for v in data_json["values"]])

            elif provider == "finnhub":
                start_ts = int(start_date.timestamp())
                end_ts = int(end_date.timestamp())
                url = f"https://finnhub.io/api/v1/stock/candle?symbol={symbol}&resolution=D&from={start_ts}&to={end_ts}&token={CONFIG['FINNHUB_KEY']}"
                response = requests.get(url)
                data_json = response.json()
                if "s" not in data_json or data_json["s"] == "no_data":
                    msg = f"No data available for {symbol} from Finnhub"
                    print(msg)
                    logging.warning(msg)
                    return None
                data = pd.DataFrame({
                    'date': [datetime.utcfromtimestamp(ts) for ts in data_json["t"]],
                    'open': data_json["o"],
                    'high': data_json["h"],
                    'low': data_json["l"],
                    'close': data_json["c"],
                    'volume': data_json["v"]
                })

            else:
                raise ValueError(f"Unknown API provider: {provider}")

            last_stock_api_call_time = time.time()

            if data.empty:
                msg = f"Empty or invalid data received for {symbol} from {provider}"
                print(msg)
                logging.warning(msg)
                return None

            data.set_index('date', inplace=True)
            msg = f"Successfully fetched {len(data)} days of stock data for {symbol} from {provider}"
            print(msg)
            logging.info(msg)
            return data

        except Exception as e:
            msg = f"Attempt {attempt} failed for {symbol} from {provider}: {str(e)}"
            print(msg)
            logging.error(msg)
            if attempt < max_attempts:
                wait_time = 4 * (2 ** (attempt - 1))
                msg = f"Retrying in {wait_time} seconds..."
                print(msg)
                logging.info(msg)
                time.sleep(wait_time)
            else:
                msg = f"Max retries reached for {symbol} from {provider}"
                print(msg)
                logging.error(msg)
                return None

def calculate_risk_metrics(data: pd.DataFrame, asset_type: str = 'stock') -> Dict:
    try:
        if data is None or data.empty:
            msg = f"No data provided for {asset_type} risk calculation"
            print(msg)
            logging.warning(msg)
            return {}

        returns = data['close'].pct_change().dropna()

        if len(returns) < 20:
            msg = f"Insufficient data for {asset_type} risk calculation"
            print(msg)
            logging.warning(msg)
            return {}

        volatility = returns.std() * np.sqrt(252)
        downside_returns = returns[returns < 0]
        downside_vol = downside_returns.std() * np.sqrt(252) if len(downside_returns) > 0 else volatility

        avg_return = returns.mean() * 252
        risk_free_rate = CONFIG['RISK_FREE_RATE']
        excess_return = avg_return - risk_free_rate

        sharpe_ratio = excess_return / volatility if volatility != 0 else 0
        sortino_ratio = excess_return / downside_vol if downside_vol != 0 else 0

        cum_returns = (1 + returns).cumprod()
        rolling_max = cum_returns.expanding().max()
        drawdowns = cum_returns / rolling_max - 1
        max_drawdown = drawdowns.min()

        var_95 = np.percentile(returns, 5)
        var_99 = np.percentile(returns, 1)
        es_95 = returns[returns <= var_95].mean()
        es_99 = returns[returns <= var_99].mean()

        skewness = stats.skew(returns)
        kurtosis = stats.kurtosis(returns)

        volume_ma = data['volume'].rolling(window=20).mean()
        volume_ratio = data['volume'].iloc[-1] / volume_ma.iloc[-1] if not volume_ma.empty and volume_ma.iloc[-1] != 0 else 1

        msg = f"{asset_type.capitalize()} risk metrics calculated successfully"
        print(msg)
        logging.info(msg)
        return {
            'volatility': volatility,
            'avg_annual_return': avg_return,
            'excess_return': excess_return,
            'sharpe_ratio': sharpe_ratio,
            'sortino_ratio': sortino_ratio,
            'max_drawdown': max_drawdown,
            'var_95': var_95,
            'var_99': var_99,
            'es_95': es_95,
            'es_99': es_99,
            'skewness': skewness,
            'kurtosis': kurtosis,
            'volume_ratio': volume_ratio
        }

    except Exception as e:
        msg = f"Error calculating {asset_type} risk metrics: {str(e)}"
        print(msg)
        logging.error(msg)
        return {}

async def analyze_crypto_pair(pair: Dict, pair_number: int, total_pairs: int) -> Optional[Dict]:
    if not CONFIG['CRYPTO_ON']:
        return None
    symbol = pair['symbol']
    try:
        msg = f"\nAnalyzing crypto pair {pair_number}/{total_pairs}: {symbol}..."
        print(msg)
        logging.info(msg)

        data = await get_crypto_data(pair, CONFIG['CRYPTO_DAYS'])
        if data is None or len(data) < 20:
            msg = f"Insufficient data for {symbol}, skipping."
            print(msg)
            logging.warning(msg)
            return None

        risk_metrics = calculate_risk_metrics(data, 'crypto')

        if not risk_metrics:
            msg = f"Failed to calculate risk metrics for {symbol}, skipping."
            print(msg)
            logging.warning(msg)
            return None

        price_change = ((data['close'].iloc[-1] - data['close'].iloc[0]) / data['close'].iloc[0]) * 100

        score = 0
        score += min(risk_metrics['sharpe_ratio'] * 10, 30)
        score += min(risk_metrics['sortino_ratio'] * 5, 20)
        score += max(-risk_metrics['max_drawdown'] * 100, 20)  # Fixed syntax error
        score += min(-risk_metrics['var_95'] * 100, 10)
        score += min(risk_metrics['volume_ratio'] * 5, 10)
        score += min(risk_metrics['excess_return'] * 50, 10)

        recommendation = "Strong Buy" if score > 80 else "Buy" if score > 60 else "Hold"

        signal_duration = 30
        if recommendation in ["Strong Buy", "Buy"]:
            momentum_factor = min(price_change / 10, 2)
            volatility_factor = max(0.5, min(2, 1/risk_metrics['volatility']))
            volume_trend = risk_metrics['volume_ratio'] / 2
            signal_duration = int(30 * momentum_factor * volatility_factor * volume_trend)
            signal_duration = max(15, min(90, signal_duration))

        msg = f"\nAnalysis complete for {symbol}:"
        msg += f"\n- Recommendation: {recommendation}"
        msg += f"\n- Score: {score:.1f}/100"
        msg += f"\n- Current Price: ${data['close'].iloc[-1]:.2f}"
        msg += f"\n- Price Change: {price_change:.1f}%"
        if recommendation in ["Strong Buy", "Buy"]:
            msg += f"\n- Recommended Holding Period: {signal_duration} days"
        msg += "\n- Key Metrics:"
        msg += f"\n  - Sharpe Ratio: {risk_metrics['sharpe_ratio']:.2f}"
        msg += f"\n  - Sortino Ratio: {risk_metrics['sortino_ratio']:.2f}"
        msg += f"\n  - 95% VaR: {risk_metrics['var_95']:.1%}"
        msg += f"\n  - Max Drawdown: {risk_metrics['max_drawdown']:.1%}"
        msg += f"\n  - Volume Ratio: {risk_metrics['volume_ratio']:.1f}x"
        msg += f"\n  - Annual Volatility: {risk_metrics['volatility']:.1%}"
        msg += "\n" + "-" * 80
        print(msg)
        logging.info(msg)

        return {
            'symbol': symbol,
            'current_price': data['close'].iloc[-1],
            'price_change': price_change,
            'recommendation': recommendation,
            'signal_duration': signal_duration,
            'risk_metrics': risk_metrics,
            'score': score,
            'date': datetime.now().strftime("%Y-%m-%d")
        }

    except Exception as e:
        msg = f"Error analyzing crypto {symbol}: {str(e)}"
        print(msg)
        logging.error(msg)
        return None

async def analyze_crypto_pairs() -> List[Optional[Dict]]:
    if not CONFIG['CRYPTO_ON']:
        return [None] * len(CONFIG['CRYPTO_PAIRS'])
    total_pairs = len(CONFIG['CRYPTO_PAIRS'])
    tasks = [
        analyze_crypto_pair(pair, i + 1, total_pairs)
        for i, pair in enumerate(CONFIG['CRYPTO_PAIRS'])
    ]
    return await asyncio.gather(*tasks)

def analyze_stock(symbol: str, stock_number: int, total_stocks: int) -> Optional[Dict]:
    try:
        msg = f"\nAnalyzing stock {stock_number}/{total_stocks}: {symbol}..."
        print(msg)
        logging.info(msg)

        data = get_stock_data(symbol)
        if data is None or len(data) < 20:
            msg = f"Insufficient data for {symbol}, skipping."
            print(msg)
            logging.warning(msg)
            return None

        risk_metrics = calculate_risk_metrics(data, 'stock')

        if not risk_metrics:
            msg = f"Failed to calculate risk metrics for {symbol}, skipping."
            print(msg)
            logging.warning(msg)
            return None

        price_change = ((data['close'].iloc[-1] - data['close'].iloc[0]) / data['close'].iloc[0]) * 100

        score = 0
        score += min(risk_metrics['sharpe_ratio'] * 10, 30)
        score += min(risk_metrics['sortino_ratio'] * 5, 20)
        score += max(-risk_metrics['max_drawdown'] * 100, 20)
        score += min(-risk_metrics['var_95'] * 100, 10)
        score += min(risk_metrics['volume_ratio'] * 5, 10)
        score += min(risk_metrics['excess_return'] * 50, 10)

        recommendation = "Strong Buy" if score > 80 else "Buy" if score > 60 else "Hold"

        signal_duration = 30
        if recommendation in ["Strong Buy", "Buy"]:
            momentum_factor = min(price_change / 10, 2)
            volatility_factor = max(0.5, min(2, 1/risk_metrics['volatility']))
            volume_trend = risk_metrics['volume_ratio'] / 2
            signal_duration = int(30 * momentum_factor * volatility_factor * volume_trend)
            signal_duration = max(15, min(90, signal_duration))

        msg = f"\nAnalysis complete for {symbol}:"
        msg += f"\n- Recommendation: {recommendation}"
        msg += f"\n- Score: {score:.1f}/100"
        msg += f"\n- Current Price: ${data['close'].iloc[-1]:.2f}"
        msg += f"\n- Price Change: {price_change:.1f}%"
        if recommendation in ["Strong Buy", "Buy"]:
            msg += f"\n- Recommended Holding Period: {signal_duration} days"
        msg += "\n- Key Metrics:"
        msg += f"\n  - Sharpe Ratio: {risk_metrics['sharpe_ratio']:.2f}"
        msg += f"\n  - Sortino Ratio: {risk_metrics['sortino_ratio']:.2f}"
        msg += f"\n  - 95% VaR: {risk_metrics['var_95']:.1%}"
        msg += f"\n  - Max Drawdown: {risk_metrics['max_drawdown']:.1%}"
        msg += f"\n  - Volume Ratio: {risk_metrics['volume_ratio']:.1f}x"
        msg += f"\n  - Annual Volatility: {risk_metrics['volatility']:.1%}"
        msg += "\n" + "-" * 80
        print(msg)
        logging.info(msg)

        return {
            'symbol': symbol,
            'current_price': data['close'].iloc[-1],
            'price_change': price_change,
            'recommendation': recommendation,
            'signal_duration': signal_duration,
            'risk_metrics': risk_metrics,
            'score': score,
            'date': datetime.now().strftime("%Y-%m-%d")
        }

    except Exception as e:
        msg = f"Error analyzing stock {symbol}: {str(e)}"
        print(msg)
        logging.error(msg)
        return None

def explain_recommendation(asset: Dict, asset_type: str = 'stock') -> str:
    metrics = asset['risk_metrics']
    reasons = [f"{asset_type.capitalize()} Analysis for {asset['symbol']} (Score: {asset['score']:.1f})"]

    if asset['recommendation'] in ["Strong Buy", "Buy"]:
        reasons.extend([
            f"Risk-adjusted returns: Sharpe Ratio = {metrics['sharpe_ratio']:.2f}",
            f"Downside protection: Sortino Ratio = {metrics['sortino_ratio']:.2f}",
            f"Risk profile: 95% VaR = {metrics['var_95']:.1%}",
            f"Resilience: Max Drawdown = {metrics['max_drawdown']:.1%}",
            f"Trading activity: Volume Ratio = {metrics['volume_ratio']:.1f}x",
            f"Returns: Excess Return = {metrics['excess_return']:.1%}",
            f"Momentum: Price Change = {asset['price_change']:.1f}%",
            f"Recommended holding period: {asset['signal_duration']} days"
        ])

        reasons.extend([
            f"\nAdvanced Risk Metrics:",
            f"- Expected Shortfall (95%): {metrics['es_95']:.1%}",
            f"- Expected Shortfall (99%): {metrics['es_99']:.1%}",
            f"- Annual Volatility: {metrics['volatility']:.1%}",
            f"- Return Distribution: Skewness = {metrics['skewness']:.2f}, Kurtosis = {metrics['kurtosis']:.2f}"
        ])
    else:
        reasons.append("Asset does not meet criteria for Buy recommendation.")

    return "\n".join(reasons)

def explain_recommendation_criteria() -> str:
    """Generate a detailed explanation of recommendation criteria and metrics."""
    explanation = "\n" + "="*80 + "\n"
    explanation += "How We Identify High-Quality Stocks:\n"
    explanation += "="*80 + "\n"
    explanation += "Our analysis identifies 'Buy' and 'Strong Buy' recommendations based on a score out of 100:\n"
    explanation += "- Strong Buy: Score > 80 - Exceptional risk-adjusted returns and momentum.\n"
    explanation += "- Buy: Score 60-80 - Solid performance with good potential.\n"
    explanation += "- Hold: Score < 60 - Does not meet criteria for immediate investment.\n\n"

    explanation += "Recommended Holding Period:\n"
    explanation += "For each 'Buy' or 'Strong Buy', we suggest a holding period (15-90 days) to maximize returns.\n"
    explanation += "This is calculated using:\n"
    explanation += "- Momentum: Recent price change (higher change = longer hold, capped at 2x).\n"
    explanation += "- Volatility: Lower volatility extends the period (0.5x to 2x).\n"
    explanation += "- Volume Trend: Higher recent volume vs. 20-day average extends the period (up to 2x).\n"
    explanation += "Default is 30 days, adjusted dynamically.\n\n"

    explanation += "Key Statistical Metrics for High-Quality Stocks:\n"
    explanation += "We use these metrics to compute the score (max contribution in parentheses):\n"
    explanation += "- Sharpe Ratio (30): Measures risk-adjusted return (higher is better).\n"
    explanation += "- Sortino Ratio (20): Focuses on downside risk-adjusted return (higher is better).\n"
    explanation += "- Max Drawdown (20): Largest historical drop (smaller is better, scored negatively).\n"
    explanation += "- 95% VaR (10): 5% worst-case daily loss (smaller is better, scored negatively).\n"
    explanation += "- Volume Ratio (10): Recent volume vs. 20-day average (higher indicates interest).\n"
    explanation += "- Excess Return (10): Annual return above risk-free rate (5%, higher is better).\n"
    explanation += "A stock is 'high-quality' if it balances strong returns with controlled risk.\n"
    explanation += "="*80
    return explanation

def print_success_indicators(count: int, asset_type: str = 'stock'):
    msg = "\n" + "="*80 + "\n"
    msg += "✓"*5 + f" {count} {asset_type.upper()} STRONG BUY/BUY CANDIDATE(S) FOUND! " + "✓"*5 + "\n"
    msg += "="*80 + "\n"
    print(msg)
    logging.info(msg)

def print_recommendation_history():
    if not stock_recommendation_history:
        msg = "\nNo previous stock recommendations."
        print(msg)
        logging.info(msg)
    else:
        msg = "\n" + "="*80 + "\n"
        msg += "Previous Stock Recommendations History:\n"
        msg += "="*80
        print(msg)
        logging.info(msg)

        for date, stocks in sorted(stock_recommendation_history.items()):
            msg = f"\nDate: {date}"
            print(msg)
            logging.info(msg)
            for i, stock in enumerate(stocks, 1):
                msg = (f"{i}. {stock['symbol']} - ${stock['current_price']:.2f} "
                       f"({stock['recommendation']}, Suggested Holding Period: {stock['signal_duration']} days)")
                print(msg)
                logging.info(msg)
        msg = "="*80
        print(msg)
        logging.info(msg)

def main():
    global stock_api_call_count, stock_api_call_window_start, all_stock_strong_buys, analyzed_symbols
    print("Starting continuous market analysis...")
    crypto_status = "enabled" if CONFIG['CRYPTO_ON'] else "disabled"
    print(f"Searching for {CONFIG['TARGET_STOCKS']} high-quality stocks (Strong Buy/Buy) using {CONFIG['STOCK_API_PROVIDER']}, crypto analysis {crypto_status}...")
    print("Logging to stock_analysis.log")
    print("=" * 80)
    logging.info(f"Starting continuous market analysis with {CONFIG['STOCK_API_PROVIDER']}, crypto analysis {crypto_status}")

    total_stocks = len(CONFIG['STOCK_SYMBOLS'])
    break_taken = False

    while len(all_stock_strong_buys) < CONFIG['TARGET_STOCKS']:
        print_recommendation_history()

        msg = f"\nStarting new analysis run on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}..."
        msg += f"\nAnalyzing up to {total_stocks} stocks (Target: {CONFIG['TARGET_STOCKS']} high-quality stocks, Found: {len(all_stock_strong_buys)} so far)"
        if CONFIG['CRYPTO_ON']:
            msg += f" and {len(CONFIG['CRYPTO_PAIRS'])} crypto pairs..."
        else:
            msg += " (crypto analysis disabled)..."
        print(msg)
        logging.info(msg)

        current_date = datetime.now().strftime("%Y-%m-%d")
        stock_strong_buys = []
        stock_counts = {'Strong Buy': 0, 'Buy': 0, 'Hold': 0, 'Failed': 0}

        remaining_symbols = [s for s in CONFIG['STOCK_SYMBOLS'] if s not in analyzed_symbols]
        if not remaining_symbols:
            msg = "\nExhausted stock list. Resetting analyzed symbols to search again..."
            print(msg)
            logging.info(msg)
            analyzed_symbols.clear()
            remaining_symbols = CONFIG['STOCK_SYMBOLS']

        for i, symbol in enumerate(remaining_symbols, 1):
            current_time = time.time()
            if current_time - stock_api_call_window_start >= 60:
                stock_api_call_count = 0
                stock_api_call_window_start = current_time

            if stock_api_call_count >= CONFIG['STOCK_API_CALLS_PER_MINUTE']:
                wait_time = 60 - (current_time - stock_api_call_window_start)
                if wait_time > 0:
                    msg = f"Stock rate limit reached, waiting {wait_time:.1f} seconds..."
                    print(msg)
                    logging.info(msg)
                    time.sleep(max(0, wait_time - (time.time() - current_time)))
                stock_api_call_count = 0
                stock_api_call_window_start = time.time()

            stock = analyze_stock(symbol, i, len(remaining_symbols))
            stock_api_call_count += 1
            analyzed_symbols.add(symbol)

            if stock is None:
                stock_counts['Failed'] += 1
                continue

            stock_counts[stock['recommendation']] += 1

            if stock['recommendation'] in ["Strong Buy", "Buy"]:
                stock_strong_buys.append(stock)
                all_stock_strong_buys.append(stock)
                print_success_indicators(len(all_stock_strong_buys), 'stock')

                msg = f"\nStock Candidate {len(all_stock_strong_buys)}: {stock['symbol']}"
                msg += f"\nCurrent Price: ${stock['current_price']:.2f}"
                msg += f"\nPrice Change: {stock['price_change']:.1f}%"
                msg += f"\nRecommendation: {stock['recommendation']}"
                msg += f"\nRecommended Holding Period: {stock['signal_duration']} days"
                msg += "\nWhy we recommend this stock:"
                msg += f"\n{explain_recommendation(stock, 'stock')}"
                msg += "\n" + "-" * 80
                print(msg)
                logging.info(msg)

            # Check for break after 10 matches
            if len(all_stock_strong_buys) == 10 and not break_taken:
                msg = f"\nReached 10 high-quality stocks. Taking a 60-second break..."
                print(msg)
                logging.info(msg)
                time.sleep(60)
                break_taken = True

            if len(all_stock_strong_buys) >= CONFIG['TARGET_STOCKS']:
                break

        if stock_strong_buys:
            stock_recommendation_history[current_date] = stock_strong_buys

        if len(stock_recommendation_history) > CONFIG['MAX_HISTORY_DAYS']:
            oldest_date = min(stock_recommendation_history.keys())
            del stock_recommendation_history[oldest_date]

        # Enhanced Run Summary
        msg = "\n" + "="*80
        msg += f"\nRun Summary for {current_date}:"
        msg += "\nStocks:"
        msg += f"\n- Total Analyzed This Run: {len(remaining_symbols)}"
        msg += f"\n- Strong Buy This Run: {stock_counts['Strong Buy']}"
        msg += f"\n- Buy This Run: {stock_counts['Buy']}"
        msg += f"\n- Hold This Run: {stock_counts['Hold']}"
        msg += f"\n- Failed This Run: {stock_counts['Failed']}"
        msg += f"\n- Total High-Quality Stocks Found: {len(all_stock_strong_buys)}/{CONFIG['TARGET_STOCKS']}"
        if stock_strong_buys:
            msg += "\n- Recommended Stocks This Run:"
            for j, s in enumerate(stock_strong_buys, 1):
                msg += f"\n  {j}. {s['symbol']} - ${s['current_price']:.2f} ({s['recommendation']})"
                msg += f" - We suggest holding this stock for {s['signal_duration']} days to optimize returns."
        else:
            msg += "\n- No Buy/Strong Buy stocks found this run."
        msg += "\nCryptos: Analysis disabled."
        msg += "\n" + "="*80
        print(msg)
        logging.info(msg)

        if len(all_stock_strong_buys) < CONFIG['TARGET_STOCKS']:
            msg = f"\nFound {len(all_stock_strong_buys)}/{CONFIG['TARGET_STOCKS']} high-quality stocks. Continuing search..."
            print(msg)
            logging.info(msg)
            msg = "Waiting 60 seconds before next run..."
            print(msg)
            logging.info(msg)
            time.sleep(60)
        else:
            # Enhanced Final Success Message
            msg = f"\nSuccess! Found {len(all_stock_strong_buys)}/{CONFIG['TARGET_STOCKS']} high-quality stocks:"
            for i, stock in enumerate(all_stock_strong_buys, 1):
                msg += f"\n{i}. {stock['symbol']} - ${stock['current_price']:.2f} ({stock['recommendation']})"
                msg += f" - We suggest holding this stock for {stock['signal_duration']} days to optimize returns based on its momentum, volatility, and volume trends."
            msg += explain_recommendation_criteria()
            print(msg)
            logging.info(msg)
            break

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        msg = "Analysis stopped by user."
        print(msg)
        logging.info(msg)
    except Exception as e:
        msg = f"Fatal error: {str(e)}"
        print(msg)
        logging.error(msg)

Starting continuous market analysis...
Searching for 15 high-quality stocks (Strong Buy/Buy) using polygon, crypto analysis disabled...
Logging to stock_analysis.log

No previous stock recommendations.

Starting new analysis run on 2025-04-11 02:48:30...
Analyzing up to 361 stocks (Target: 15 high-quality stocks, Found: 0 so far) (crypto analysis disabled)...

Analyzing stock 1/361: AAPL...
Attempt 1: Fetching stock data for AAPL from polygon (2025-03-12 to 2025-04-11)...
Successfully fetched 21 days of stock data for AAPL from polygon
Stock risk metrics calculated successfully

Analysis complete for AAPL:
- Recommendation: Hold
- Score: -23.8/100
- Current Price: $198.85
- Price Change: -8.4%
- Key Metrics:
  - Sharpe Ratio: -1.14
  - Sortino Ratio: -1.82
  - 95% VaR: -7.4%
  - Max Drawdown: -23.0%
  - Volume Ratio: 2.6x
  - Annual Volatility: 77.0%
--------------------------------------------------------------------------------

Analyzing stock 2/361: MSFT...

Waiting 60.0 seconds fo